In [3]:
import json

from utils.read_file import *
from utils.model_tool import get_ans
from utils.split_law import split_law_to_token


In [4]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"
FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"
FINANCE_PATH = "競賽資料集/reference/finance"
FILE_NUM = 2000

In [ ]:

def save_predictions_to_json(predictions, filename="pred_retrieve.json"):#Modified
    output = {"answers": predictions}
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=4, ensure_ascii=False)
    print(f"Predictions saved to {filename}")

def gernerate_faq_ans(embbeded_model, question, i: int, results: list):
    faq = read_json(FAQ_PATH)
    document = read_target_faq(faq, question[i]["source"])
    k_highest, _ = get_ans(embbeded_model, question[i], document)
    predict = question[i]["source"][k_highest[0]]
    results.append({"qid": i + 1, "retrieve": predict})  # Modified
        
    
    

def gernerate_insurance_ans(embedded_model, question, i: int, results: list):

    already_read = ["" for _ in range(FILE_NUM)]

    src = np.array(question[i]["source"])
    document = read_target_insurance_pdf(INSURANCE_PATH, src, already_read)
    tokens = []
    chunk_real_index = []

    for j in range(len(document)):
        chunks = split_law_to_token(document[j])
        tokens.extend(chunks)
        chunk_real_index.extend([src[j] for k in range(len(chunks))])

    chunk_real_index = np.array(chunk_real_index)
    token_index, _ = get_ans(embedded_model, question[i], tokens, 10)
    all_predict = chunk_real_index[token_index]

    predict = all_predict[0]

    results.append({"qid": i + 1, "retrieve": predict})  #modified



In [ ]:
results = []

save_predictions_to_json(results)